<a href="https://colab.research.google.com/github/Hamza-Ali0237/PyTorch-RCNN-Cats-And-Dogs/blob/main/RCNN-PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset: [https://www.kaggle.com/datasets/tongpython/cat-and-dog](https://)

In [ ]:
import torch as T
import torch.nn as nn
import torchvision as tv
from torchvision.models import vgg16, VGG16_Weights

In [ ]:
class ObjectDetectorCNN(nn.Module):
  def __init__(self):
    super(ObjectDetectorCNN, self).__init__()
    # Define VGG16 model as backbone
    vgg = vgg16(weights=VGG16_Weights.DEFAULT)
    self.backbone = nn.Sequential(*list(
        vgg.features.children()
    ))
    # Extract input_features from VGG16
    input_features = nn.Sequential(*list(
        vgg.classifier.children()
    ))[0].in_features
    # Define the classifier
    self.classifier = nn.Sequential(
        nn.Linear(input_features, 512),
        nn.ReLU(),
        nn.Linear(512, 2)
    )
    # Define the regressor
    self.box_regressor = nn.Sequential(
        nn.Linear(input_features, 32),
        nn.ReLU(),
        nn.Linear(32, 4)
    )

  def forward(self, x):
    features = self.backbone(x)
    bboxes = self.regressor(features)
    classes = self.classifier(features)
    return bboxes, classes